In [1]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
import time

c:\Users\user\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)


In [6]:
# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_date, end_date, start_pg=1, end_pg=6):
    # 날짜 형식 검사
    try:
        assert len(start_date.split(".")) == 3 and len(end_date.split(".")) == 3
    except:
        print("날짜 형식이 잘못되었습니다. YYYY.MM.DD 형식으로 입력해주세요.")
        return
    
    # URL의 날짜 부분 생성
    date_part = "&pd=3&ds=" + start_date + "&de=" + end_date
    
    urls = []
    for i in range(start_pg, end_pg + 1):
        page = makePgNum(i)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + date_part + "&sort=2&start=" + str(page)
        urls.append(url)
    
    if len(urls) == 1:
        print("생성url: ", urls[0])
        return urls[0]
    else:
        print("생성url: ", urls)
        return urls


In [7]:

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    time.sleep(1)
    return url


In [8]:

#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
print("\n크롤링할 시작 날짜: ",start_date)   
#검색 종료할 페이지 입력
end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")
print("\n크롤링할 종료 날짜: ",end_date)   


# naver url 생성
url = makeUrl(search,start_date,end_date)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    time.sleep(1)  # 1초 대기. 필요에 따라 조절 가능.\n
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass


크롤링할 시작 날짜:  2017.01.01

크롤링할 종료 날짜:  2017.01.31
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=암호화폐&pd=3&ds=2017.01.01&de=2017.01.31&sort=2&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=암호화폐&pd=3&ds=2017.01.01&de=2017.01.31&sort=2&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=암호화폐&pd=3&ds=2017.01.01&de=2017.01.31&sort=2&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=암호화폐&pd=3&ds=2017.01.01&de=2017.01.31&sort=2&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=암호화폐&pd=3&ds=2017.01.01&de=2017.01.31&sort=2&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=암호화폐&pd=3&ds=2017.01.01&de=2017.01.31&sort=2&start=51']


100%|██████████| 97/97 [01:37<00:00,  1.01s/it]


In [9]:
# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    # content = news_html.select("div#dic_area")
    # if content == []:
    #     content = news_html.select("#articeBody")
    content = news_html.select("#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
    

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

# print("검색된 기사 갯수: 총 ",(end_pg+1-start_pg)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기##
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

100%|██████████| 39/39 [00:28<00:00,  1.35it/s]



[뉴스 제목]
['[2017 국제 핫이슈]블록체인 기술 확산', '[신년기획]블록체인, 금융권 넘어 IT기업도 관심', '블록체인 등 최신 핀테크 기술 도입… `금융 한류` 일으키자', '비트코인, 3년만에 처음으로 1000달러 돌파', "'암호화 화폐' 비트코인, 3년만에 1천달러 돌파", 'KISA, 암호기술팀·블록체인TF 신설', '비트코인 1000달러 넘었다', '김기춘 위기에 빠트린 사건 세 가지의 결말', '비트코인 사상최고가…中 자본통제에 `풍선효과`', '돈 대신 인질 요구하는 랜섬웨어, 범행방식 바뀐 이유', '[2017년 신년기획] 강해진 랜섬웨어,  ‘인질극’은 계속된다', '비트코인 큰손은 중국?… 반나절새 위안화↗비트코인↘ 배후설', "'아직도 보험청구서 작성하세요?'…블록체인 기술로 OK", '사이버 공격 시달리는 EU…지난해 EU본부 해킹시도 20% 늘어', '선거철 EU, 해킹 주의보…"러시아 조심"', '비트코인·페이팔·텔레그램…테러지원 3종세트', '비트코인·페이팔 "테러지원에 활용되고 있다"', '“블록체인, 4차 산업혁명의 고속도로 역할 할 것”', '작년 랜섬웨어 피해 1438건… KISA "1년새 86.8% 늘어"', '"비트코인 IS 테러자금으로 사용"', '비트코인·페이팔 테러지원 수단 악용', '경기도정에 블록체인 기술 접목…온라인 직접 투표 기대', '[브리핑] 돌아온 반기문, 불 붙은 조기대선 국면', "[데이터 '稅'상] [카드뉴스]'비트코인' 화폐로 인정한 일본, '소비세' 안매긴다", '"中 비트코인, 인민은행·투자자 둘 다 주목해야"', '새해 서점가 미래예측서 줄줄이 출간', '[리더스카페] 수수료 없는 세상이 온다, 블록체인 현실화', "미래부, 블록체인 연구개발 '시동'", '[책마을] 마음 놓고 P2P거래…수수료 없는 세상이 온다', "중국 당국이 실행 가능한 '6가지' 자본 유출 대응 방안", '&lt;블록체인혁명&gt; 미래 금융산업의 핵심이 될 분산형 장부', '해킹·수수료 없는 P2P…블록

In [10]:
#데이터 프레임 저장
filename = '{}_{}_{}.csv'.format(search, start_date.replace(".", ""), end_date.replace(".", ""))
news_df.to_csv(filename, encoding='utf-8-sig', index=False)